## Identificación de capas disponibles en un servidor WFS

El presente código sirve para poder identificar, desde el geoserver de la SEDATU, las capas disponibles en el servidor WFS.

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os

In [2]:
#url_xml="https://ide.sedatu.gob.mx/geoserver/ows?service=WMS&version=1.3.0&request=GetCapabilities"
url_xml2="https://ide.sedatu.gob.mx/geoserver/ows?service=WFS&version=1.0.0&request=GetCapabilities"

In [3]:
response = requests.get(url_xml2)

In [4]:
#Extraer el contenido del xml
root = ET.fromstring(response.content)
root

<Element '{http://www.opengis.net/wfs}WFS_Capabilities' at 0x000001E9D8001F80>

In [5]:
namespace = {"wfs": "http://www.opengis.net/wfs"}

layers = []
for layer in root.findall(".//wfs:FeatureType", namespaces=namespace):
    name = layer.find("wfs:Name", namespaces=namespace)
    title = layer.find("wfs:Title", namespaces=namespace)
    url = layer.find("wfs:OnlineResource", namespaces=namespace)
    if name is not None and title is not None:
        layers.append((name.text, title.text))
        
print("Número de capas en el geoserver:", len(layers))

Número de capas en el geoserver: 3067


In [6]:
#Llevar la lista a una dataframe
df = pd.DataFrame(layers, columns=["nombre", "titulo"])
df

,nombre,titulo
0,geonode:00_ageb_cpyv_inegi_2010,00_ageb_cpyv_inegi_2010
1,geonode:00_ageb_cpyv_inegi_2020,00_ageb_cpyv_inegi_2020
2,geonode:00_ageb_eqmmovilidad_inegi_2020,00_ageb_eqmmovilidad_inegi_2020
3,geonode:00_ageb_pcuadrasconrampas_2020,00_ageb_pcuadrasconrampas_2020
4,geonode:00_ageb_pobdiurnaynocturna_inegi_2022,00_ageb_pobdiurnaynocturna_inegi_2022
...,...,...
3062,geonode:zprimaria_c590cec20e9dfdc782212d38dd80...,zprimaria_c590cec20e9dfdc782212d38dd803e71
3063,geonode:zs_0904c250eaa9ce223d35dcdf9dd3b9a3,zs_0904c250eaa9ce223d35dcdf9dd3b9a3
3064,geonode:zsecundaria,zsecundaria
3065,geonode:zsecundaria_,zsecundaria_


In [7]:
## Identificar cuáles capas corresponden al SITU

url_capas="https://situ.sedatu.gob.mx/wa/publico/getCapas/"
response = requests.get(url_capas)
response.status_code

200

In [8]:
capas_situ=pd.DataFrame(response.json()).rename(columns=str.lower)
#Dejar solo la columna titulo
capas_situ=capas_situ[["nombre_capa"]]
capas_situ

,nombre_capa
0,geonode:a__27_EV_INEGI_20200
1,geonode:a__00_IGPOC_CENAPRED_2019
2,geonode:EA_03_20067_Zonificacion_Secundaria_gr...
3,geonode:a__06_EV_INEGI_2020
4,geonode:a__22_MUN_INEGI_2020
...,...
2010,geonode:a__10_2_07101_Zonificacion_secundaria_...
2011,geonode:a__13_SOLARES_RAN_2020
2012,geonode:a__13_ENT_INEGI_2020
2013,geonode:a__00_MUN_SCINCE_2010


In [9]:
#Corroborar si las capas del SITU están en la lista de capas del geoserver
capas_finales=pd.merge(capas_situ,df,how="inner",left_on="nombre_capa",right_on="nombre")
print("Diferencia entre capas encontradas en el SITU y las disponibles en el geoserver:",len(capas_situ)-len(capas_finales))

Diferencia entre capas encontradas en el SITU y las disponibles en el geoserver: 7


In [10]:
#Listado de capas no encontradas
capas_situ[~capas_situ["nombre_capa"].isin(capas_finales["nombre_capa"])]

,nombre_capa
63,geonode:RegionesBienestar4326
174,geonode:juarez_mus_centrogeo_2022c
276,geonode:inegi_RH00_ha_4326
505,geonode:MOT_ZNVM_corte
602,geonode:a__18_INV_FM_INEGI_2016
922,geonode:a__00CatLoc_042021a
1903,Sin Información


### Crear proceso para descarga de las capas

Se crea una función para descargar las capas con la finalidad de realizar una BD local de las capas disponibles en el servidor WFS.

In [11]:
def descarga_capa(df):
    base_url = "https://ide.sedatu.gob.mx/geoserver/ows"  #url de la IDE de SEDATU

   #Proceso iterativo sobre la df 
    for _, row in df.iterrows():
        layer_name = row["nombre"]  
        layer_title = row["titulo"]  

       
        params = {
            "service": "WFS",
            "version": "2.0.0",
            "request": "GetFeature",
            "typeName": layer_name,
            "outputFormat": "application/json", 
        }

      
        response = requests.get(base_url, params=params)

       
        if response.status_code == 200:
          
            filename = f"{layer_title}.geojson"
            with open(filename, "w", encoding="utf-8") as f:
                f.write(response.text)
            print(f"Capa descargada '{layer_title}' como '{filename}'")
        else:
            print(f"Fallo al intentar descargar capa '{layer_title}'. Status code: {response.status_code}")

In [12]:
#Ejemplo: muestra de 1
descarga_capa(capas_finales.sample(1))

Capa descargada 'Localidades urbanas y rurales amanzanadas Mexico Marco Geoestadistico 2016' como 'Localidades urbanas y rurales amanzanadas Mexico Marco Geoestadistico 2016.geojson'


In [13]:
os.getcwd()

'C:\\Users\\claud\\Documents\\GitHub\\situ'